In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

ONTOLOGY_SEQ_DATASET_FP = "../data/intermediary/drosophila_full_protein_ontology_and_seqs.csv"

# load sequence and ontology data into memory
df_original = pd.read_csv(ONTOLOGY_SEQ_DATASET_FP)
relevant_subset = df_original[df_original.qualifier.isin(["enables", "involved_in"])].dropna()
df = pd.DataFrame(index=relevant_subset.seq.unique(), columns=relevant_subset.go_name.unique()).fillna(0)
for _, row in relevant_subset.iterrows():
    df.loc[row.seq, row.go_name] = 1

In [ ]:
def det_class_balance(df_):
    rows, cols = df_.shape
    class_balance = df_.sum().sum() / (rows * cols)
    return class_balance

In [ ]:
thresh2balance = {}
n_terms2balance = []
nterms2thresh = []
for thresh in range(100,500,10):
    interesting_go_names = [
        name for (name, freq)
        in relevant_subset.go_name.value_counts().to_dict().items()
        if thresh < freq
    ]
    bal = det_class_balance(df.loc[:,interesting_go_names])
    thresh2balance[thresh] = (bal, len(interesting_go_names))
    n_terms2balance.append((len(interesting_go_names), bal))
    nterms2thresh.append((len(interesting_go_names), thresh))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10,5))
ax1.plot(*zip(*n_terms2balance))
ax1.set(xlabel="number of terms", ylabel="class balance")
y, x = zip(*nterms2thresh)
ax2.hist(x,y)
ax2.set(ylabel="number of terms", xlabel="threshold")